In [19]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from ckonlpy.tag import Twitter
from konlpy.tag import Okt
import pandas as pd

import nltk
import os
import re


In [31]:
twitter = Twitter()
okt = Okt()

In [32]:
def tokenize_doc(tokenized):
    
    for i in range(1,5):
        name = str(i)+'.txt'
        data_cleansing_file = open(name,'r',-1,'utf-8')           # -1 은 버퍼 / utf-8 추가 or 파일 인코딩 ANSI 변경으로 인코딩 문제 해결 가능
        data = data_cleansing_file.readlines()

        data_word = []
        
        for j in range(len(data)):
            try:
                data_word.append(okt.nouns(data[j]))

            except Exception as e:
                continue

        data_cleansing_file.close()
        tokenized_doc.append(data_word)

In [33]:
tokenized_doc = []

tokenize_doc(tokenized_doc)

In [35]:
print(tokenized_doc)

[[['법원', '배터리', '자료', '내야', '이번', '기술', '유출', '대립'], ['전기차', '배터리', '영업비밀', '유출', '여부', '송전', '돌입', '화학', '과', '이노베이션', '이', '이번', '소송', '기술', '유출', '가능성', '대립', '이노베이션', '배터리', '핵심', '기술', '자료', '로펌', '미국', '법원', '제출', '경쟁', '국', '유출', '수', '고', '주장', '화학', '외부', '유출', '가능성', '전혀', '고', '두', '회사', '소송', '위해', '관련', '기술', '자료', '해외', '정부', '승인', '정부', '승인', '지도', '주목'], [], ['화학', '지난달', '미국', '국제무역', '위원회', '델라웨어', '지방법원', '이노베이션', '명의', '핵심', '인력', '등', '영업', '비밀', '침해', '며', '소송', '이노베이션', '공개', '채용', '통해', '자발', '지원', '경력', '직', '후보자', '중', '채용', '며', '반박'], [], ['전기차', '배터리', '소송', '위해', '화학', '세계', '최대', '규모', '로펌', '덴톤스', '선임', '이노베이션', '미국', '대형', '로펌', '코빙턴', '앤드', '벌링', '법률', '대리인', '선임'], [], ['배터리', '기술', '입증', '위해', '해외', '정부', '승인'], [], ['문제', '소송', '진행', '화학', '이노베이션', '정부', '검토', '영업비밀', '관련', '자료', '로펌', '법원', '제출', '점', '현재', '화학', '정부', '승인', '위', '절차', '상황'], [], ['산업', '통상', '자원부', '관계자', '이노베이션', '화학', '전기차', '배터리', '기술', '국가', '핵심', '기술', '지정', '보호', '관련', '정보',

In [29]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

TypeError: decoding to str: need a bytes-like object, int found

In [89]:
#필요한 모듈 임포트하기
import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
from konlpy.tag import Okt
okt = Okt()
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from wordcloud import WordCloud
from datetime import datetime

In [92]:
id2word=corpora.Dictionary(tokenized_doc)
id2word.filter_extremes(no_below = 2) #20회 이하로 등장한 단어는 삭제
texts = data_word
corpus=[id2word.doc2bow(text) for text in texts]


os.environ['MALLET_HOME'] = 'C:/Users/MoTS/topic_modeling/mallet-2.0.8/'

mallet_path = 'C:/Users/MoTS/topic_modeling/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=2, id2word=id2word)

TypeError: decoding to str: need a bytes-like object, list found

In [72]:
topics = ldamallet.print_topics(num_words=10)

In [73]:
for topic in topics:
    print(topic)

(0, '0.155*"기술" + 0.109*"유출" + 0.101*"화학" + 0.047*"미국" + 0.047*"승인" + 0.047*"영업비밀" + 0.039*"수출" + 0.039*"고" + 0.031*"로펌" + 0.031*"산업"')
(1, '0.097*"소송" + 0.062*"이노베이션" + 0.056*"보호" + 0.056*"정부" + 0.049*"법원" + 0.049*"자료" + 0.042*"제출" + 0.035*"과정" + 0.035*"관련" + 0.035*"배터리"')


In [81]:
from gensim.models import LdaModel

def mallet_to_lda(optimal_model):
    model_gensim = LdaModel(
        id2word=optimal_model.id2word, num_topics=optimal_model.num_topics,
        alpha=optimal_model.alpha, eta=0, iterations=1000,
        gamma_threshold=0.001,
        dtype=np.float32
    )
    model_gensim.sync_state()
    model_gensim.state.sstats = optimal_model.wordtopics
    return model_gensim
model = mallet_to_lda(optimal_model)

import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.294569  0.048804       1        1  12.688849
3     -0.004553 -0.335876       2        1  12.677895
0      0.210693  0.054382       3        1  12.555975
4     -0.140863 -0.098099       4        1  12.544669
5      0.207023  0.143949       5        1  12.539402
2     -0.099831 -0.208761       6        1  12.397169
7     -0.203502  0.207079       7        1  12.366976
6     -0.263536  0.188522       8        1  12.229061, topic_info=   Category  Freq Term      Total  loglift  logprob
0   Default  17.0   기술  17.000000     30.0     30.0
4   Default  11.0   유출  11.000000     29.0     29.0
12  Default  12.0   소송  12.000000     28.0     28.0
22  Default   8.0   정부   8.000000     27.0     27.0
27  Default  12.0   화학  12.000000     26.0     26.0
..      ...   ...  ...        ...      ...      ...
23   Topic8   0.0   제출   7.903142     -inf     -inf
24   Topic8   0.0   주장   2.234875     -inf     -inf
25   Topic8   0.0   해외   2.433663     -inf     -inf
26   Topic8   0.0   핵심   5.001755     -inf     -inf
28   Topic8   0.0   회사   1.735604     -inf     -inf

[303 rows x 6 columns], token_table=      Topic      Freq Term
term                      
7         1  0.894905  가능성
38        7  1.066292   검토
51        6  1.152337    것
52        7  1.066292   경우
8         3  1.079416    고
...     ...       ...  ...
25        4  0.410903   해외
26        7  0.999649   핵심
27        1  0.551384   화학
27        5  0.472615   화학
28        6  1.152337   회사

[86 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 5, 6, 3, 8, 7])